##### Copyright 2025 Google LLC.

In [5]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Gemini Text-to-speech
<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Get_started_TTS.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

The Gemini API can transform text input into single speaker or multi-speaker audio (podcast-like experience like in [NotebookLM](https://notebooklm.google.com/). This notebook provides an example of how to control the *Text-to-speech* (TTS) capability of the Gemini model and guide its style, accent, pace, and tone.

Before diving in the code, you should try this capability on [AI Studio](https://aistudio.google.com/prompts/new_chat?model=gemini-2.5-flash-preview-tts).

**Note that the TTS model can only do TTS, it does not have the reasoning capabilities of the Gemini models, so you can ask things like "say this in that style", but not "tell me why the sky is blue".** If that's what you want, you should use the [Live API](./Get_started_LiveAPI.ipynb) instead.

The [documentation](https://ai.google.dev/gemini-api/docs/audio-generation) is also a good place to start discovering the TTS capability.

<!-- Notice Badge -->
<table align="left" border="3">
  <tr>
    <!-- Emoji -->
    <td bgcolor="#DCE2FF">
      <font size=30>🪧</font>
    </td>
    <!-- Text Content Cell -->
    <td bgcolor="#DCE2FF">
      <h4><font color=black>Audio-out is a preview feature. It is free to use for now with quota limitations, but is subject to change.</font></h4>
    </td>
  </tr>
</table>

## Setup

### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](../quickstarts/Authentication.ipynb) for an example.

In [6]:
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

### Install and initialize the SDK


In [7]:
!pip install -U -q "google-genai>=1.16.0" # 1.16 is needed for multi-speaker audio


In [8]:
from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)

### Select a model

Audio-out is only supported by the "`tts`" models, `gemini-2.5-flash-preview-tts` and `gemini-2.5-pro-preview-tts`.

For more information about all Gemini models, check the [documentation](https://ai.google.dev/gemini-api/docs/models/gemini) for extended information on each of them.


In [9]:
MODEL_ID = "gemini-2.5-flash-preview-tts" # @param ["gemini-2.5-flash-preview-tts","gemini-2.5-pro-preview-tts"] {"allow-input":true, isTemplate: true}

Next create a helper function to prompt the model and play back the audio in the notebook:

In [10]:
# @title Helper functions (just run that cell)

import contextlib
import wave
from IPython.display import Audio

file_index = 0

@contextlib.contextmanager
def wave_file(filename, channels=1, rate=24000, sample_width=2):
    with wave.open(filename, "wb") as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(sample_width)
        wf.setframerate(rate)
        yield wf

def play_audio_blob(blob):
  global file_index
  file_index += 1

  fname = f'audio_{file_index}.wav'
  with wave_file(fname) as wav:
    wav.writeframes(blob.data)

  return Audio(fname, autoplay=True)

def play_audio(response):
    return play_audio_blob(response.candidates[0].content.parts[0].inline_data)

## Generate a simple audio output

Let's start with something simple:

In [11]:
response = client.models.generate_content(
  model=MODEL_ID,
  contents="Say 'hello, my name is Gemini!'",
  config={"response_modalities": ['Audio']},
)

The generated ouput is in the response `inline_data` and as you can see it's indeed audio data.

In [12]:
blob = response.candidates[0].content.parts[0].inline_data
print(blob.mime_type)

audio/L16;codec=pcm;rate=24000


To be able to listen to the generated audio in colab, you're going to use our helper function to write the output in a file and play it.

In [13]:
play_audio_blob(blob)

Note that the model can only do TTS, so you should always tell it to "say", "read", "TTS" something, otherwise it won't do anything.

## Control how the model speaks

There are 30 different built-in voices you can use and 24 supported languages which gives you plenty of combinations to try.

### Choose a voice

Choose a voice among the 30 different ones. You can find their characteristics in the [documentation](https://ai.google.dev/gemini-api/docs/speech-generation#voices).

In [23]:
voice_name = "Despina" # @param ["Zephyr", "Puck", "Charon", "Kore", "Fenrir", "Leda", "Orus", "Aoede", "Callirhoe", "Autonoe", "Enceladus", "Iapetus", "Umbriel", "Algieba", "Despina", "Erinome", "Algenib", "Rasalgethi", "Laomedeia", "Achernar", "Alnilam", "Schedar", "Gacrux", "Pulcherrima", "Achird", "Zubenelgenubi", "Vindemiatrix", "Sadachbia", "Sadaltager", "Sulafar"]

In [25]:
response = client.models.generate_content(
  model=MODEL_ID,
  contents="""Sagen Sie auf deutsch: „Ich bin ein sehr kenntnisreiches Modell, insbesondere im Umgang mit Erdung“, warten Sie 3 Sekunden und sagen Sie dann: „Finden Sie nicht?“".""",
  config={
      "response_modalities": ['Audio'],
      "speech_config": {
          "voice_config": {
              "prebuilt_voice_config": {
                  "voice_name": Gacrux
              }
          }
      }
  },
)

play_audio(response)

### Change the language

Just tell the model to speak in a certain language and it will. The [documentation](https://ai.google.dev/gemini-api/docs/speech-generation#languages) lists all the supported ones.

In [16]:
response = client.models.generate_content(
  model=MODEL_ID,
  contents="""
    Read this in French:

    Les chaussettes de l'archiduchesse sont-elles sèches ? Archi-sèches ?
    Un chasseur sachant chasser doit savoir chasser sans son chien.
  """,
  config={"response_modalities": ['Audio']},
)

play_audio(response)

### Prompt the model to speak in certain ways

You can control style, tone, accent, and pace using natural language prompts, for example:

In [17]:
response = client.models.generate_content(
  model=MODEL_ID,
  contents="""
    Say in an spooky whisper:
    "By the pricking of my thumbs...
    Something wicked this way comes!"
  """,
  config={"response_modalities": ['Audio']},
)

play_audio(response)

In [18]:
response = client.models.generate_content(
  model=MODEL_ID,
  contents="""
    Read this disclaimer in as fast a voice as possible while remaining intelligible:

    [The author] assumes no responsibility or liability for any errors or omissions in the content of this site.
    The information contained in this site is provided on an 'as is' basis with no guarantees of completeness, accuracy, usefulness or timeliness
  """,
  config={"response_modalities": ['Audio']},
)

play_audio(response)

## Mutlti-speakers

The TTS model can also read discussions between 2 speakers (like [NotebookLM](https://Fnotebooklm.google.com) podcast feature). You just need to tell it that there are two speakers:

In [19]:
response = client.models.generate_content(
  model=MODEL_ID,
  contents="""
    Make Speaker1 sound tired and bored, and Speaker2 sound excited and happy:

    Speaker1: So... what's on the agenda today?
    Speaker2: You're never going to guess!
  """,
  config={"response_modalities": ['Audio']},
)

play_audio(response)

ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.5-flash-tts'}, 'quotaValue': '3'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '55s'}]}}

You can also select the voices for each participants and pass their names to the model.

But first let's generate a discussion between two scientists:

In [ ]:
transcript = client.models.generate_content(
    model='gemini-2.5-flash',
    contents="""
      Hi, please generate a short (like 100 words) transcript that reads like
      it was clipped from a podcast by excited herpetologists, Dr. Claire and
      her assistant, the young Aurora.
    """
  ).text

print(transcript)

Then let's have the TTS model render the conversation using the voices you want.

In [ ]:
config = types.GenerateContentConfig(
    response_modalities=["AUDIO"],
    speech_config=types.SpeechConfig(
        multi_speaker_voice_config=types.MultiSpeakerVoiceConfig(
            speaker_voice_configs=[
                types.SpeakerVoiceConfig(
                    speaker='Dr. Claire',
                    voice_config=types.VoiceConfig(
                        prebuilt_voice_config=types.PrebuiltVoiceConfig(
                            voice_name='sulafat',
                        )
                    )
                ),
                types.SpeakerVoiceConfig(
                    speaker='Aurora',
                    voice_config=types.VoiceConfig(
                        prebuilt_voice_config=types.PrebuiltVoiceConfig(
                            voice_name='Leda',
                        )
                    )
                ),
            ]
        )
    )
)

response = client.models.generate_content(
  model=MODEL_ID,
  contents="TTS the following conversation between a very excited Dr. Claire and her assistant, the young Aurora: "+transcript,
  config=config,
)

play_audio(response)

# What's next?

Now that you know how to generate multi-speaker conversations, here are other cool things to try:
*   Instead of speech, learn how to generate music conversation using the [Lyria RealTime](./Get_started_LyriaRealTime.ipynb),
*   Discover how to generate [images](./Get_started_imagen.ipynb) or [videos](./Get_started_Veo.ipynb),
*   Instead of generation music or audio, find out how to Gemini can [understand Audio files](./Audio.ipynb),
*   Have a real-time conversation with Gemini using the [Live API](./Get_started_LiveAPI.ipynb).